In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd
from tqdm import tqdm

# ——— 参数配置 —————————————————————————————————————————————————————————————
patch_npy = "/home/SSD_248/zhaolx/Unet_train/global/class/300_phase.npy"
grid_npz  = "/home/SSD_248/zhaolx/Dataset/grid_data.npz"
test_csv  = "/home/SSD_248/zhaolx/Code/SmArtUnetERA5/dataset_path/all/test_dataset_2022.csv"
out_npy   = "/home/SSD_248/zhaolx/Unet_train/global/class/global_acc.npy"

# ——— 加载 patch 结果 ———————————————————————————————————————————————————————
# Shape: (N, 2, 64, 64)
patches = np.load(patch_npy)
N, C, H, W = patches.shape
assert C == 2 and H == 64 and W == 64, "patch_npy 的 shape 应为 (N,2,64,64)"

# ——— 加载全局经纬度网格 —————————————————————————————————————————————————————
g = np.load(grid_npz)
lat_grid = g["lat_grid"]   # (2000, 5143)
lon_grid = g["lon_grid"]   # (2000, 5143)
R, S = lat_grid.shape

# ——— 读入所有 patch 对应的文件路径（确保顺序一致）————————————————————————————————
paths = pd.read_csv(test_csv)["path"].tolist()
assert len(paths) == N, f"paths 长度 {len(paths)} 和 patches 的 N={N} 不一致"

# ——— 准备输出数组，用 -999 填充——————————————————————————————————————————————
global_acc = np.full((N, 2, R, S), -999, dtype=patches.dtype)

# ——— 预计算 1D 向量，便于逐像素最近点查找————————————————————————————————————
# 每行纬度相同，所以取任意一列
lat_vec = lat_grid[:, 0]    # (2000,)
# 每列经度相同，所以取任意一行
lon_vec = lon_grid[0, :]    # (5143,)

# ——— 主循环：对每个样本，加载其 .npz，取出 patch 的经纬度，映射到大网格 ——————————————
for i, fp in enumerate(tqdm(paths, desc="Mapping patches → global")):
    ds = np.load(fp)
    # 在你的 dataset 中，modis[5] 是纬度，modis[6] 是经度
    lat_patch = ds["modis"][5]  # (64,64)
    lon_patch = ds["modis"][6]  # (64,64)

    # 找到每个 patch 像素对应全局网格的行索引：shape (64,64)
    rows = np.abs(lat_vec[:, None, None] - lat_patch[None, :, :]).argmin(axis=0)
    # 找到每个 patch 像素对应全局网格的列索引：shape (64,64)
    cols = np.abs(lon_vec[None, :, None] - lon_patch[None, :, :]).argmin(axis=1)

    # 将 patches[i, 0/1, :, :] 散射到 global_acc[i, c, :, :]
    for c in (0, 1):
        global_acc[i, c, rows, cols] = patches[i, c]

# ——— 保存最终结果————————————————————————————————————————————————————————
np.save(out_npy, global_acc)
print(f"✅ 保存完成：{out_npy} （shape = {global_acc.shape}）")